In [ ]:
pip install qiskit==2.1

In [ ]:
pip install qiskit_aer==0.17

In [ ]:
pip install qiskit_ibm_runtime==0.40

In [ ]:
pip install numpy==2.2

In [ ]:
pip install pylatexenc==2.10

In [ ]:
pip install matplotlib==3.10

# Chapter 14. Quantum Error Correction and Fault Tolerance

## 14.3. Implementing the Shor code in Qiskit

In [1]:
from qiskit import QuantumCircuit

In [2]:
def ShorCode(qc, noise):
    qc.barrier()
    
    # Encoding
    qc.cx(0, 3)
    qc.cx(0, 6)
    for qubit in [0,3,6]:
        qc.h(qubit)
        qc.cx(qubit, qubit + 1)
        qc.cx(qubit, qubit + 2)

    # Noisy channel
    qc.barrier()
    noise(qc)
    qc.barrier()

    # Error correction
    for qubit in [0,3,6]:
        qc.cx(qubit, qubit + 1)
        qc.cx(qubit, qubit + 2)
        qc.ccx(qubit + 2, qubit + 1, qubit)
        qc.h(qubit)
    qc.cx(0, 3)
    qc.cx(0, 6)
    qc.ccx(6, 3, 0)


In [3]:
qc = QuantumCircuit(9)                 # Initialize the circuit.
qc.h(0)                                # Prepare the input state.
ShorCode(qc, lambda qc: qc.z(2))    # Apply the code with noise.

In [4]:
qc.draw(output="mpl")
draw = qc.draw(output="mpl")
draw.savefig("qiskit-shorc.pdf")

In [5]:
from qiskit.quantum_info import Statevector
statevector = Statevector(qc)

In [6]:
from numpy import isclose

for index, value in enumerate(statevector):
    if not isclose(value, 0):
        # Use bin to get a binary string for the index.
        # Remove the first two characters (which are '0b').
        index_string = bin(index)[2:]

        # Add leading zeros.
        index_string = index_string.zfill(9)

        # Flip the string (to account for Qiskit's conventions).
        index_string = index_string[::-1]
        
        print(f"{value} |{index_string}>")

(0.7071067811865471+0j) |000100100>
(0.7071067811865471+0j) |100100100>


In [7]:
qc = QuantumCircuit(9)
qc.h(0)
# Here we use a different error:
ShorCode(qc, lambda qc: qc.ry(4,3))
statevector = Statevector(qc)

for index, value in enumerate(statevector):
    if not isclose(value, 0):
        index_binary = bin(index)[2:]
        index_string = index_binary.zfill(9)
        index_string = index_string[::-1]
        print(f"{value} |{index_string}>")

(-0.294260250091814+0j) |000000000>
(-0.294260250091814+0j) |100000000>
(0.6429703766239176+0j) |000111000>
(0.6429703766239176+0j) |100111000>
